In [1]:
import refexps
from os import listdir, path
from itertools import combinations
from collections import defaultdict
import numpy as np
from scipy import stats

In [5]:
### OVERALL FORM COORDINATION ###

from refexps import load_data, move_level, get_previous, gamerun_level
from os import path, listdir

import numpy as np


data_folder = 'Dataset/Push_to_Talk_Corpus/RefExp'
games = listdir(data_folder)

total_n = 0
sameform_n = 0
sameform_game_n = 0
ignore = 0

for curgame in games:
    curgame = path.join(data_folder, curgame)
    data = load_data(curgame)
    info_move = move_level(data)
    one = info_move[1]
    for move in info_move:
        for o in move[1]:
            obj = move[1][o]
            for ref in obj:
                total_n +=1
                #print i 
                if get_previous(ref, obj) == 'no_previous':
                    ignore += 1
                else: 
                    if ref[3] == get_previous(ref, obj)[3]:
                        sameform_n +=1
                    
print total_n, sameform_n
ntotal_n = total_n - ignore
print float(sameform_n) / ntotal_n


4263 327
0.466476462197


In [34]:
### FORM COORDINATION AT MOVE LEVEL ###
import numpy as np

from refexps import load_data, move_level, get_previous, gamerun_level
from os import path

all_move_data = []
probability_distribution = []

data_folder = 'Dataset/Push_to_Talk_Corpus/RefExp'
games = listdir(data_folder)
entropy = []

for curgame in games:
    curgame = path.join(data_folder, curgame)
    data = load_data(curgame)
    info_move = move_level(data)
    for move in info_move:
        for o in move[1]:
            move_total_n = 0
            move_sameform_n = 0
            obj = move[1][o]
            moves_forms = defaultdict(int)
            for i in obj:
                curform = i[3]
                moves_forms[curform] +=1
                move_total_n +=1
                if i[3] == get_previous(i, obj)[3]:
                    move_sameform_n +=1
            sum_forms = sum(moves_forms.values())
            for form in moves_forms:
                moves_forms[form] = float(moves_forms[form])/float(sum_forms)
            moves_forms = moves_forms.values()
            entropy.append(stats.entropy(moves_forms))
            move_final_n = move_total_n - 1 #necessary because we don't count the first refexp
            if move_final_n != 0:
                move_value = float(move_sameform_n) / move_final_n
                #print move_final_n 
                all_move_data.append(move_value)

print np.mean(all_move_data)
print np.var(all_move_data)
print np.mean(entropy)

0.112842810972
0.0606617688538
0.332317760146


In [20]:
### FORM COORDINATION AT GAME LEVEL ###
import numpy as np

from refexps import load_data, move_level, get_previous, gamerun_level
from os import path

all_game_data = []

data_folder = 'Dataset/Push_to_Talk_Corpus/RefExp'
games = listdir(data_folder)
ignore = 0
for curgame in games:
    game_total = 0
    game_sameform = 0
    curgame = path.join(data_folder, curgame)
    data = load_data(curgame)
    info_game = gamerun_level(data)
    for obj in info_game:
        game_total +=1
        r = info_game[obj]
        for ref in r:
            if ref[3] == get_previous(ref, r)[3]:
                game_sameform +=1
        game_final = game_total - 1
        if game_final !=0:
            game_value = float(game_sameform) / game_final
            all_game_data.append(game_value)
                    
            
print np.mean(all_game_data)
print np.var(all_game_data)

0.59342646014
0.179222928573


In [21]:
### MOVE LEVEL ###

move_level_align = {'FTT': [], 'PTT': []}
move_level_variation = {'FTT': [], 'PTT': []}

playermove_level_align_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 
playermove_level_variation_speaker= {'FTT': defaultdict(list), 'PTT': defaultdict(list)}  


dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'

for dialogue in listdir(dataset_dir):
    dial_type = dialogue[:3]
    dialogue = path.join(dataset_dir, dialogue)
    dialogue = refexps.load_data(dialogue)
    #move level
    dialogue_move = refexps.move_level(dialogue)
    for move in dialogue_move:
        refs = move[1]
        for obj in refs:
            align_speaker = {}
            total_n = len(refs[obj])
            sameform_n =0
            ignore = 0
            entropy_speaker = defaultdict(int)
            total_n_speaker = defaultdict(int)
            sameform_n_speaker = defaultdict(int)
            ignore_speaker = defaultdict(int)
            obj_forms_speaker = defaultdict(dict)
            obj_forms = defaultdict(int)
            for ref in refs[obj]:
                obj_forms[ref[3]] += 1
                speaker = ref[1]
                if not obj_forms_speaker[speaker].has_key(ref[3]):
                    obj_forms_speaker[speaker][ref[3]] = 0
                obj_forms_speaker[speaker][ref[3]] += 1
                align_speaker[speaker] = 0
                total_n_speaker[speaker] +=1
                if get_previous(ref, refs[obj]) == 'no_previous':
                        ignore += 1
                        ignore_speaker[speaker] += 1
                else: 
                    if ref[3] == get_previous(ref, refs[obj])[3]:
                        sameform_n +=1
                        sameform_n_speaker[speaker] += 1
            sum_forms = sum(obj_forms.values())
            for form in obj_forms:
                obj_forms[form] = float(obj_forms[form])/float(sum_forms)
            obj_forms = obj_forms.values()
            variation = stats.entropy(obj_forms)
            #movelevel variation and alignment (for each object)
            ntotal_n = total_n - ignore
            if ntotal_n != 0:
                align = float(sameform_n) / ntotal_n
                move_level_align[dial_type].append(align)
            for s in align_speaker:
                ntotal_n = total_n_speaker[s] - ignore_speaker[s]
                if ntotal_n != 0:
                    align = float(sameform_n_speaker[s]) / ntotal_n 
                    playermove_level_align_speaker[dial_type][s].append(align)
            move_level_variation[dial_type].append(variation)
            variation_speaker = {}
            for s in obj_forms_speaker:
                sum_forms = sum(obj_forms_speaker[s].values())
                for form in obj_forms_speaker[s]:
                    obj_forms_speaker[s][form] = float(obj_forms_speaker[s][form])/float(sum_forms)
                obj_forms_speaker[s] = obj_forms_speaker[s].values()
                variation_speaker[s] = stats.entropy(obj_forms_speaker[s])
            for s in variation_speaker:
                playermove_level_variation_speaker[dial_type][s].append(np.mean(variation_speaker[s]))
                
move_level_align_all = []
move_level_variation_all = []

playermove_level_align_speaker_all = defaultdict(list)
playermove_level_variation_speaker_all = defaultdict(list)

results_move = {'FTT': {}, 'PTT': {}, 'ALL': defaultdict(list)}

for dial_type in playermove_level_variation_speaker:
    move_level_align_all += move_level_align[dial_type]
    move_level_variation_all += move_level_variation[dial_type]
    playermove_level_variation = []
    for s in playermove_level_variation_speaker[dial_type]:
        playermove_level_variation += playermove_level_variation_speaker[dial_type][s]
        playermove_level_variation_speaker_all[s] += playermove_level_variation_speaker[dial_type][s]
        results_move[dial_type]['Move variation '+ s] = playermove_level_variation_speaker[dial_type][s]
        results_move['ALL']['Move variation '+ s].extend(playermove_level_variation_speaker[dial_type][s])
    results_move[dial_type]['Move players variation'] = playermove_level_variation
    results_move[dial_type]['Move variation'] = move_level_variation[dial_type]
    results_move['ALL']['Move players variation'].extend(playermove_level_variation)
    results_move['ALL']['Move variation'].extend(move_level_variation[dial_type])
    playermove_level_align = []
    for s in playermove_level_align_speaker[dial_type]:
        playermove_level_align += playermove_level_align_speaker[dial_type][s]
        playermove_level_align_speaker_all[s] += playermove_level_align_speaker[dial_type][s]
        results_move[dial_type]['Move alignment '+ s] = playermove_level_align_speaker[dial_type][s]
        results_move['ALL']['Move alignment '+ s].extend(playermove_level_align_speaker[dial_type][s])
    results_move['ALL']['Move players alignment'].extend(playermove_level_align)
    results_move['ALL']['Move alignment'].extend(move_level_align[dial_type])
    results_move[dial_type]['Move players alignment'] = playermove_level_align
    results_move[dial_type]['Move alignment'] = move_level_align[dial_type]
for d_type in results_move:
    print d_type
    for value in results_move[d_type]:
        dataset = results_move[d_type][value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tVariance: '+ str(np.var(dataset))

ALL
	Move variation: 
	Mean: 0.332317760146	Variance: 0.1750816849
	Move alignment e-utts: 
	Mean: 0.45975224038	Variance: 0.217110883214
	Move alignment p-utts: 
	Mean: 0.443759512938	Variance: 0.218943806194
	Move variation e-utts: 
	Mean: 0.208745145699	Variance: 0.1308447937
	Move players variation: 
	Mean: 0.247577157772	Variance: 0.133609834926
	Move players alignment: 
	Mean: 0.45260447036	Variance: 0.217993309288
	Move variation p-utts: 
	Mean: 0.265903376528	Variance: 0.133867262775
	Move alignment: 
	Mean: 0.46748444831	Variance: 0.198087106491
PTT
	Move variation: 
	Mean: 0.278553031808	Variance: 0.149992257751
	Move alignment e-utts: 
	Mean: 0.446607669617	Variance: 0.217336083048
	Move alignment p-utts: 
	Mean: 0.487972508591	Variance: 0.22465488126
	Move variation e-utts: 
	Mean: 0.185025396055	Variance: 0.127888896251
	Move players variation: 
	Mean: 0.193832787852	Variance: 0.108137185647
	Move players alignment: 
	Mean: 0.465714285714	Variance: 0.221141950113
	Move var

In [61]:
values = results_move['FTT'].keys()
print 'DIFFERENCES PTT AND FTT'
for value in values:
    ttest =  stats.ttest_ind(results_move['FTT'][value],results_move['PTT'][value] )
    if ttest[1] < 0.05:
        print value
        print str(ttest)+'\tSignificantly different'
print 'DIFFERENCES E-UTTS AND P-UTTS'
ttest = stats.ttest_ind(results_move['FTT']['Move variation e-utts'], results_move['FTT']['Move variation p-utts'])
if ttest[1] < 0.05:
    print 'Move variation speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['PTT']['Move variation e-utts'], results_move['PTT']['Move variation p-utts'])
if ttest[1] < 0.05:
    print 'Move variation speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['FTT']['Move alignment e-utts'], results_move['FTT']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_move['PTT']['Move alignment e-utts'], results_move['PTT']['Move alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers PTT: '+ str(ttest) + '\tSignificantly different'

DIFFERENCES PTT AND FTT
Move variation
Ttest_indResult(statistic=3.4850558669230662, pvalue=0.00051395974742639088)	Significantly different
Move players variation
Ttest_indResult(statistic=4.5782820353006315, pvalue=5.1432189710479544e-06)	Significantly different
Move variation p-utts
Ttest_indResult(statistic=4.7812717285301289, pvalue=2.044482197778199e-06)	Significantly different
DIFFERENCES E-UTTS AND P-UTTS
Move variation speakers FTT: Ttest_indResult(statistic=-2.9650167295201713, pvalue=0.0031246313958449832)	Significantly different


In [22]:
### GAME LEVEL 

game_level_align = {'FTT': [], 'PTT': []}
game_level_variation = {'FTT': [], 'PTT': []}
playergame_level_align_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 
playergame_level_variation_speaker = {'FTT': defaultdict(list), 'PTT': defaultdict(list)} 


dataset_dir = 'Dataset/Push_to_Talk_Corpus/RefExp'

for dialogue in listdir(dataset_dir):
    dial_type = dialogue[:3]
    dialogue = refexps.load_data(path.join(dataset_dir, dialogue))
    dialogue_game = refexps.gamerun_level(dialogue)
    for obj in dialogue_game:
        refobj = dialogue_game[obj]
        sim_align = []
        sim_align_speaker = defaultdict(list)
        sim_variation_speaker = defaultdict(list)
        sim_variation = []
        align_speaker = {}
        total_n = len(refobj)
        sameform_n =0
        ignore = 0
        entropy_speaker = defaultdict(int)
        total_n_speaker = defaultdict(int)
        sameform_n_speaker = defaultdict(int)
        ignore_speaker = defaultdict(int)
        obj_forms_speaker = defaultdict(dict)
        obj_forms = defaultdict(int)
        for ref in refobj:
            obj_forms[ref[3]] += 1
            speaker = ref[1]
            if not obj_forms_speaker[speaker].has_key(ref[3]):
                obj_forms_speaker[speaker][ref[3]] = 0
            obj_forms_speaker[speaker][ref[3]] += 1
            align_speaker[speaker] = 0
            total_n_speaker[speaker] +=1
            if get_previous(ref, refobj) == 'no_previous':
                    ignore += 1
                    ignore_speaker[speaker] += 1
            else: 
                if ref[3] == get_previous(ref, refobj)[3]:
                    sameform_n +=1
                    sameform_n_speaker[speaker] += 1
        sum_forms = sum(obj_forms.values())
        for form in obj_forms:
            obj_forms[form] = float(obj_forms[form])/float(sum_forms)
        obj_forms = obj_forms.values()
        variation = stats.entropy(obj_forms)
        #movelevel variation and alignment (for each object)
        ntotal_n = total_n - ignore
        if ntotal_n != 0:
            align = float(sameform_n) / ntotal_n
            game_level_align[dial_type].append(align)
        for s in align_speaker:
            ntotal_n = total_n_speaker[s] - ignore_speaker[s]
            if ntotal_n != 0:
                align = float(sameform_n_speaker[s]) / ntotal_n 
                playergame_level_align_speaker[dial_type][s].append(align)
        game_level_variation[dial_type].append(variation)
        variation_speaker = {}
        for s in obj_forms_speaker:
            sum_forms = sum(obj_forms_speaker[s].values())
            for form in obj_forms_speaker[s]:
                obj_forms_speaker[s][form] = float(obj_forms_speaker[s][form])/float(sum_forms)
            obj_forms_speaker[s] = obj_forms_speaker[s].values()
            variation_speaker[s] = stats.entropy(obj_forms_speaker[s])
        for s in variation_speaker:
            playergame_level_variation_speaker[dial_type][s].append(np.mean(variation_speaker[s]))

game_level_align_all = []
game_level_variation_all = []

playergame_level_align_speaker_all = defaultdict(list)
playergame_level_variation_speaker_all = defaultdict(list)

results_game = {'FTT': {}, 'PTT': {}, 'ALL': defaultdict(list)}

for dial_type in playergame_level_variation_speaker:
    game_level_align_all += game_level_align[dial_type]
    game_level_variation_all += game_level_variation[dial_type]
    playergame_level_variation = []
    for s in playergame_level_variation_speaker[dial_type]:
        playergame_level_variation += playergame_level_variation_speaker[dial_type][s]
        playergame_level_variation_speaker_all[s] += playergame_level_variation_speaker[dial_type][s]
        results_game[dial_type]['game variation '+ s] = playergame_level_variation_speaker[dial_type][s]
        results_game['ALL']['game variation '+ s].extend(playergame_level_variation_speaker[dial_type][s])
    results_game[dial_type]['game players variation'] = playergame_level_variation
    results_game[dial_type]['game variation'] = game_level_variation[dial_type]
    results_game['ALL']['game players variation'].extend(playergame_level_variation)
    results_game['ALL']['game variation'].extend(game_level_variation[dial_type])
    playergame_level_align = []
    for s in playergame_level_align_speaker[dial_type]:
        playergame_level_align += playergame_level_align_speaker[dial_type][s]
        playergame_level_align_speaker_all[s] += playergame_level_align_speaker[dial_type][s]
        results_game[dial_type]['game alignment '+ s] = playergame_level_align_speaker[dial_type][s]
        results_game['ALL']['game alignment '+ s].extend(playergame_level_align_speaker[dial_type][s])
    results_game['ALL']['game players alignment'].extend(playergame_level_align)
    results_game['ALL']['game alignment'].extend(game_level_align[dial_type])
    results_game[dial_type]['game players alignment'] = playergame_level_align
    results_game[dial_type]['game alignment'] = game_level_align[dial_type]
for d_type in results_game:
    print d_type
    for value in results_game[d_type]:
        dataset = results_game[d_type][value]
        print '\t'+ value + ': '
        print '\tMean: '+ str(np.mean(dataset)) +'\tVariance: '+ str(np.var(dataset))

ALL
	game alignment p-utts: 
	Mean: 0.449735780837	Variance: 0.188758965793
	game alignment: 
	Mean: 0.444805602004	Variance: 0.155068157702
	game variation: 
	Mean: 0.534019661065	Variance: 0.202876874191
	game players alignment: 
	Mean: 0.439131141677	Variance: 0.184511626525
	game alignment e-utts: 
	Mean: 0.429814916621	Variance: 0.180594731877
	game players variation: 
	Mean: 0.426089519053	Variance: 0.182723079869
	game variation p-utts: 
	Mean: 0.452261614225	Variance: 0.177344225487
	game variation e-utts: 
	Mean: 0.37024111513	Variance: 0.18962025601
PTT
	game alignment p-utts: 
	Mean: 0.486840904944	Variance: 0.209444053026
	game alignment: 
	Mean: 0.444857338351	Variance: 0.15871517704
	game variation: 
	Mean: 0.436468236812	Variance: 0.175992116823
	game players alignment: 
	Mean: 0.452642688281	Variance: 0.199412665757
	game alignment e-utts: 
	Mean: 0.423184818482	Variance: 0.188896595105
	game players variation: 
	Mean: 0.317694260129	Variance: 0.150143233173
	game varia

In [24]:
values = results_game['FTT'].keys()
print 'DIFFERENCES PTT AND FTT'
for value in values:
    ttest =  stats.ttest_ind(results_game['FTT'][value],results_game['PTT'][value] )
    if ttest[1] < 0.05:
        print value
        print str(ttest)+'\tSignificantly different'
print 'DIFFERENCES E-UTTS AND P-UTTS'
ttest = stats.ttest_ind(results_game['FTT']['game variation e-utts'], results_game['FTT']['game variation p-utts'])
if ttest[1] < 0.05:
    print 'Move variation speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['PTT']['game variation e-utts'], results_game['PTT']['game variation p-utts'])
if ttest[1] < 0.05:
    print 'Move variation speakers PTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['FTT']['game alignment e-utts'], results_game['FTT']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers FTT: '+ str(ttest) + '\tSignificantly different'
ttest = stats.ttest_ind(results_game['PTT']['game alignment e-utts'], results_game['PTT']['game alignment p-utts'])
if ttest[1] < 0.05:
    print 'Move alignment speakers PTT: '+ str(ttest) + '\tSignificantly different'

DIFFERENCES PTT AND FTT
game variation
Ttest_indResult(statistic=4.9782022883890198, pvalue=8.7425486916552275e-07)	Significantly different
game players variation
Ttest_indResult(statistic=6.9734844772394098, pvalue=6.8121348733274028e-12)	Significantly different
game variation p-utts
Ttest_indResult(statistic=6.9871745511694634, pvalue=8.8429419312496837e-12)	Significantly different
game variation e-utts
Ttest_indResult(statistic=2.4153043739671904, pvalue=0.016479894966994001)	Significantly different
DIFFERENCES E-UTTS AND P-UTTS
Move variation speakers FTT: Ttest_indResult(statistic=-2.9954419186139458, pvalue=0.0029191307913059177)	Significantly different
